In [1]:
import socket
import numpy as np
import time

In [2]:
HOST = 'localhost'
PORT = 5000
BLK_SIZE = 1024*16*1000
NBLK = 32
HDR_SIZE = 64

header = bytearray(HDR_SIZE)
buffer = bytearray(BLK_SIZE*2)
buffer2 = bytearray(BLK_SIZE*4)
arr = np.frombuffer(buffer, dtype='i2')
arr2 = np.frombuffer(buffer2, dtype='i2')
arr2[:] = 0
data = np.empty((1000*NBLK, 1024, 16), dtype='float32')

def readBlock(s):
    recv_size = s.recv_into(memoryview(header), HDR_SIZE)
    if recv_size != 64:
        print(f'Error receiving header, size={recv_size}')
        raise

    view = memoryview(buffer)
    next_offset = 0
    while BLK_SIZE*2 - next_offset > 0:
        recv_size = s.recv_into(view[next_offset:], BLK_SIZE*2 - next_offset)
        next_offset += recv_size
#        print(f'receive {recv_size} bytes')

    arr2[1::2] = arr

In [3]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    start = time.time()
    for n in range(NBLK):
        readBlock(s)
        data[n*1000:(n+1)*1000,:,:] = np.frombuffer(buffer2, dtype='float32').reshape((1000, 1024, 16))
        print(f'received #{n} block, counter={header[:8].hex()}')
    total = time.time() - start

print(f'number of blocks: {NBLK},  average time per block: {total / NBLK}s')
#print(f'transfer rate: {NBLK*BLK_SIZE*2 / total * 10**(-9)} GB/s')

received #0 block, counter=0d14a7db76ccbbaa
received #1 block, counter=0d49b3db76ccbbaa
received #2 block, counter=0d7ebfdb76ccbbaa
received #3 block, counter=0db3cbdb76ccbbaa
received #4 block, counter=0de8d7db76ccbbaa
received #5 block, counter=0d1de4db76ccbbaa
received #6 block, counter=0d52f0db76ccbbaa
received #7 block, counter=0d87fcdb76ccbbaa
received #8 block, counter=0dbc08dc76ccbbaa
received #9 block, counter=0df114dc76ccbbaa
received #10 block, counter=0d2621dc76ccbbaa
received #11 block, counter=0d5b2ddc76ccbbaa
received #12 block, counter=0d9039dc76ccbbaa
received #13 block, counter=0dc545dc76ccbbaa
received #14 block, counter=0dfa51dc76ccbbaa
received #15 block, counter=0f2f5edc76ccbbaa
received #16 block, counter=0f646adc76ccbbaa
received #17 block, counter=0f9976dc76ccbbaa
received #18 block, counter=0fce82dc76ccbbaa
received #19 block, counter=0f038fdc76ccbbaa
received #20 block, counter=0f389bdc76ccbbaa
received #21 block, counter=0f6da7dc76ccbbaa
received #22 block, 

In [ ]:
header.hex()

In [ ]:
header[:5].hex()